In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data):
    fft_result = fft(data)
    n = len(data)
    freq = np.fft.fftfreq(n, d=1/24)
    
    # Filtrar para mantener solo frecuencias cercanas a 1 ciclo diario
    fft_filtered = fft_result.copy()
    fft_filtered[(freq < 0.9) | (freq > 1.1)] = 0
    
    # Reconstrucción de la señal para obtener el ciclo circadiano
    return np.real(ifft(fft_filtered))
    
def extract_circadian_cycle2(data, sample_rate=24):
    # Eliminar tendencia
    detrended = data - data.mean()

    # FFT con ventana
    window = np.hanning(len(detrended))
    fft_result = fft(detrended * window)

    # Análisis espectral
    freq = np.fft.fftfreq(len(data), d=1/sample_rate)
    power = np.abs(fft_result)**2

    # Identificar frecuencia circadiana dominante
    daily_freq_mask = (freq >= 0.8) & (freq <= 1.2)
    if not np.any(daily_freq_mask):
        print("Advertencia: No se detectó ritmo circadiano claro")

    # Filtrado adaptativo
    fft_filtered = fft_result.copy()
    fft_filtered[~daily_freq_mask] *= 0.1  # Atenuar otras frecuencias

    return np.real(ifft(fft_filtered))

# Configuración de datos de entrada
# Cargar los datos
nombres_csv = {1} #{1,2,3,4,468,479,4003,4151,4160,4173}
for nombre_csv in nombres_csv:
    df= pd.read_csv(f'Data1_actividad/{nombre_csv}_act.csv')
    df = df.copy()

    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()


    # Función para procesar una columna de actividad y calcular niveles de estrés
    def process_activity(df, activity_column):
        daily_cycles = {}
        
        # Extraer ciclo circadiano para cada día y almacenarlo
        for date, group in df.groupby(df.index.date):
            if len(group) == 24:  # Verificar 24 horas de datos
                daily_cycles[date] = extract_circadian_cycle2(group[activity_column].values)

        # Calcular el ciclo circadiano promedio
        average_cycle = np.mean(list(daily_cycles.values()), axis=0)

        # Calcular distancias entre el ciclo promedio y los ciclos diarios
        hourly_distances = []
        for date, cycle in daily_cycles.items():
            for hour in range(24):
                hourly_distances.append({
                    'Fecha': date,
                    'Hora': hour,
                    #'distancia': euclidean(average_cycle, cycle),
                    'distancia': np.absolute(average_cycle[hour]-cycle[hour])
                })

        distance_df = pd.DataFrame(hourly_distances)

        # Calcular umbrales para clasificar niveles de estrés
        # min_distance = distance_df['distancia'].min()
        # max_distance = distance_df['distancia'].max()
        # threshold1 = min_distance + (max_distance - min_distance) / 2
        # threshold2 = threshold1 + (max_distance - threshold1) / 2
        # threshold3 = max_distance
        
        # Establecer umbrales de estrés usando desviación estándar
        mean_dist = distance_df['distancia'].mean()
        std_dist = distance_df['distancia'].std()
        threshold1 = mean_dist + std_dist
        threshold2 = mean_dist + 2 * std_dist
        

        # Clasificación de niveles de estrés basada en los umbrales
        def get_stress_level(distance):
            if distance <= threshold1:
                return 'normal'
            elif distance <= threshold2:
                return 'alerta'
            else:
                return 'peligro'

        # Aplicar la función de nivel de estrés a cada distancia
        distance_df['Nivel_estres'] = distance_df['distancia'].apply(get_stress_level)
        
        return distance_df[['Fecha', 'Hora', 'Nivel_estres']]

    # Procesar ambas columnas de actividad y calcular niveles de estrés
    stress_levels_1 = process_activity(df, 'activity_level').rename(columns={'Nivel_estres': 'Nivel_estres_1'})
    stress_levels_2 = process_activity(df, 'activity_level_2').rename(columns={'Nivel_estres': 'Nivel_estres_2'})

    # Fusionar niveles de estrés con el DataFrame original
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    stress_levels_1['Fecha'] = pd.to_datetime(stress_levels_1['Fecha'])
    stress_levels_2['Fecha'] = pd.to_datetime(stress_levels_2['Fecha'])
    df['Hora'] = df['Hora'].astype(int)
    stress_levels_1['Hora'] = stress_levels_1['Hora'].astype(int)
    stress_levels_2['Hora'] = stress_levels_2['Hora'].astype(int)

    # Fusionar todos los DataFrames en df_final
    df_final = df.merge(stress_levels_1, on=['Fecha', 'Hora'], how='left')
    df_final = df_final.merge(stress_levels_2, on=['Fecha', 'Hora'], how='left')

    #df_final.to_csv(f'Datos_entrenamiento/{nombre_csv}_e.csv', index=False) 
    #df_final.to_csv(f'Datos_prueba/{nombre_csv}_p.csv', index=False)   


           Fecha  Hora  distancia
0     2024-01-01     0   0.012851
1     2024-01-01     1   0.009540
2     2024-01-01     2   0.005081
3     2024-01-01     3   0.000439
4     2024-01-01     4   0.001266
...          ...   ...        ...
3931  2024-06-30    19   0.012493
3932  2024-06-30    20   0.015675
3933  2024-06-30    21   0.019310
3934  2024-06-30    22   0.026445
3935  2024-06-30    23   0.030256

[3936 rows x 3 columns]
           Fecha  Hora  distancia
0     2024-01-01     0   0.058517
1     2024-01-01     1   0.049512
2     2024-01-01     2   0.036756
3     2024-01-01     3   0.028364
4     2024-01-01     4   0.014600
...          ...   ...        ...
3931  2024-06-30    19   0.000901
3932  2024-06-30    20   0.010319
3933  2024-06-30    21   0.024111
3934  2024-06-30    22   0.027225
3935  2024-06-30    23   0.032464

[3936 rows x 3 columns]


In [7]:
df_final

,Fecha,Hora,period eating,period other,period resting,period rumination,period eating_count,period other_count,period resting_count,period rumination_count,activity_level,activity_level_2,Nivel_estres_1,Nivel_estres_2
0,2024-01-01,0,0.079,0.0,0.410,0.511,1.0,0.0,4.0,2.0,-0.209528,-1.025107,normal,normal
1,2024-01-01,1,0.000,0.0,0.652,0.348,0.0,0.0,1.0,1.0,-0.327217,-0.443815,normal,normal
2,2024-01-01,2,0.000,0.0,0.443,0.557,0.0,0.0,1.0,1.0,-0.249102,-0.443815,normal,normal
3,2024-01-01,3,0.000,0.0,0.462,0.538,0.0,0.0,2.0,2.0,-0.256203,-0.887631,normal,normal
4,2024-01-01,4,0.000,0.0,0.711,0.289,0.0,0.0,1.0,1.0,-0.349269,-0.443815,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3931,2024-06-30,19,1.000,0.0,0.000,0.000,1.0,0.0,0.0,0.0,0.261279,0.200319,normal,normal
3932,2024-06-30,20,0.341,0.0,0.000,0.659,1.0,0.0,0.0,1.0,0.034052,-0.074599,normal,normal
3933,2024-06-30,21,0.000,0.0,0.551,0.449,0.0,0.0,1.0,2.0,-0.289468,-0.718734,normal,normal
3934,2024-06-30,22,0.416,0.0,0.000,0.584,1.0,0.0,0.0,1.0,0.059913,-0.074599,normal,normal


In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt

# Función para calcular el ciclo circadiano utilizando la Transformada de Fourier
def extract_circadian_cycle(data, sample_rate=24):
    detrended = data - data.mean()  # Eliminar tendencia
    window = np.hanning(len(detrended))  # Aplicar ventana de Hann
    fft_result = fft(detrended * window)

    # Filtrar para mantener solo frecuencias cercanas a 1 ciclo diario
    freq = np.fft.fftfreq(len(data), d=1/sample_rate)
    daily_freq_mask = (freq >= 0.9) & (freq <= 1.1)
    fft_filtered = fft_result.copy()
    fft_filtered[~daily_freq_mask] = 0  # Atenuar otras frecuencias

    return np.real(ifft(fft_filtered))

# Función para calcular el cambio circadiano mediante la distancia euclidiana
def calculate_circadian_change(df, activity_column):
    # Tomar una serie temporal de 36 horas
    data_36_hours = df[activity_column].values[:36]
    
    # Dividir en las primeras 24 horas y las últimas 24 horas
    first_24_hours = data_36_hours[:24]
    last_24_hours = data_36_hours[-24:]
    
    # Extraer el ciclo circadiano para cada segmento
    first_cycle = extract_circadian_cycle(first_24_hours)
    last_cycle = extract_circadian_cycle(last_24_hours)
    
    # Alinear ambos ciclos restando sus medias
    aligned_first_cycle = first_cycle - np.mean(first_cycle)
    aligned_last_cycle = last_cycle - np.mean(last_cycle)
    
    # Calcular la distancia euclidiana entre los ciclos circadianos alineados
    circadian_distance = euclidean(aligned_first_cycle, aligned_last_cycle)
    
    # Calcular la distancia por hora para establecer umbrales de estrés
    hourly_distances = np.abs(aligned_first_cycle - aligned_last_cycle)
    mean_dist = hourly_distances.mean()
    std_dist = hourly_distances.std()
    threshold1 = mean_dist + std_dist
    threshold2 = mean_dist + 2 * std_dist
    
    # Clasificar nivel de estrés usando los umbrales de distancia
    stress_level = 'normal' if circadian_distance <= threshold1 else 'alerta' if circadian_distance <= threshold2 else 'peligro'
    
    # Crear DataFrame con los resultados de la distancia horaria y el nivel de estrés general
    results = pd.DataFrame({
        'Hora': range(24),
        'Distancia_horaria': hourly_distances,
        'Nivel_estres': ['normal' if d <= threshold1 else 'alerta' if d <= threshold2 else 'peligro' for d in hourly_distances]
    })
    
    results.loc[:, 'Distancia_total'] = circadian_distance
    results.loc[:, 'Nivel_estres_total'] = stress_level
    
    return results

# Configuración de datos de entrada
nombres_csv = {1}  # {1,2,3,4,468,479,4003,4151,4160,4173}

for nombre_csv in nombres_csv:
    df = pd.read_csv(f'Data1_actividad/{nombre_csv}_act.csv')

    # Convertir 'Fecha' y 'Hora' a datetime e indexar el DataFrame
    df['datetime'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')
    df = df.set_index('datetime')
    df = df.sort_index()
    
    # Calcular el cambio en el ciclo circadiano para cada columna de actividad
    stress_levels_1 = calculate_circadian_change(df, 'activity_level')
    stress_levels_2 = calculate_circadian_change(df, 'activity_level_2')
    
    # Mostrar resultados
    print(f'Archivo {nombre_csv} - Actividad 1:')
    print(stress_levels_1)
    print(f'\nArchivo {nombre_csv} - Actividad 2:')
    print(stress_levels_2)
    print('\n' + '-'*40 + '\n')


In [ ]:

Mis disculpas, cometí un error al no incluir explícitamente las series temporales de 36 horas y al omitir la distancia euclidiana. Ahora, adaptaré el código para trabajar con una serie temporal completa de 36 horas y para calcular la distancia euclidiana entre los ciclos circadianos de los primeros y últimos 24 horas.

Aquí está el código corregido:

Serie temporal de 36 horas: Tomará una ventana de datos de 36 horas de actividad.
Cálculo del ciclo circadiano para las primeras 24 y últimas 24 horas.
Alineación temporal de ambos ciclos.
Cálculo de la distancia euclidiana entre los ciclos circadianos alineados.
Umbrales de estrés basados en desviación estándar.
Descripción de los cambios
Serie Temporal de 36 Horas:

La función calculate_circadian_change utiliza los primeros 36 valores de la columna de actividad para la extracción de los ciclos.
Ciclo Circadiano:

Se extrae el ciclo circadiano tanto de las primeras 24 horas como de las últimas 24 horas usando extract_circadian_cycle.
Distancia Euclidiana:

La distancia euclidiana se calcula entre los ciclos alineados de las primeras y últimas 24 horas, comparándola con umbrales para determinar si el ritmo circadiano ha cambiado.
Clasificación del Estrés:

La distancia total (circadian_distance) se evalúa con los umbrales (threshold1 y threshold2), clasificando el nivel de estrés como "normal", "alerta" o "peligro".
Este ajuste asegura que se cumplan todos los requerimientos especificados, con la serie temporal de 36 horas y el cálculo de la distancia euclidiana para detectar cambios en el ritmo circadiano.